In [15]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
# Load data from a single file
def load_data(data_path):
    df = pd.read_csv(data_path)
    data = df['filename'].tolist()
    labels = df['label'].tolist()
    return data, labels

In [17]:
# Preprocess audio data
def preprocess_audio(file_path, target_duration=10):
    audio, _ = librosa.load(file_path, sr=None)

    target_length = int(target_duration * _)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]

    mfccs = librosa.feature.mfcc(y=audio, sr=_, n_mfcc=13)
    return mfccs

In [18]:
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [19]:
def train_model(X_train, y_train, X_val, y_val):
    model = build_model(input_shape=X_train.shape[1:])
    optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Data Augmentation
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Model Training
    model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=20, callbacks=[early_stopping], verbose=1)

    # Save the trained model to a file
    model.save('temp_hindi_hate_speech_model.h5')

    return model

In [20]:
# Main function
def main():

    train_data_path = 'Hindi_train.csv'
    test_data_path = 'Hindi_test.csv'

    # Load training data
    data_train, labels_train = load_data(train_data_path)

    # Load testing data
    data_test, labels_test = load_data(test_data_path)

    # Convert string labels to integers
    y_train = np.array([1 if label == 'Yes' else 0 for label in labels_train])
    y_test = np.array([1 if label == 'Yes' else 0 for label in labels_test])

    # Combine datasets for k-fold cross-validation
    data_combined = data_train + data_test
    labels_combined = y_train.tolist() + y_test.tolist()

    # Convert to numpy arrays
    X_combined = np.array([preprocess_audio(os.path.join('./audios/', filename)) for filename in tqdm(data_combined, desc="Processing Audio")])
    y_combined = np.array(labels_combined)

    # Initialize cross-validator
    n_splits = 5  # You can adjust the number of folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold = 1
    accuracies = []
    all_true_labels = []
    all_pred_labels = []

    for train_index, val_index in skf.split(X_combined, y_combined):
        X_train, X_val = X_combined[train_index], X_combined[val_index]
        y_train, y_val = y_combined[train_index], y_combined[val_index]

        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Train the model
        model = train_model(X_train, y_train, X_val, y_val)

        # Predict on the validation set
        y_pred_probs = model.predict(X_val)
        y_pred = (y_pred_probs > 0.5).astype(int)

        # Evaluate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Fold {fold} - Validation Accuracy: {accuracy * 100:.2f}%")

        # Collect true and predicted labels for confusion matrix
        all_true_labels.extend(y_val)
        all_pred_labels.extend(y_pred.flatten())

        accuracies.append(accuracy)
        fold += 1

    # Save the final trained model with date and time in the name, in 'final_model' directory
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_model_filename = f"final_model_{current_datetime}.h5"
    final_model_directory = 'final_model'
    
    os.makedirs(final_model_directory, exist_ok=True)
    final_model_path = os.path.join(final_model_directory, final_model_filename)
    
    model.save(final_model_path)
    print(f"\nFinal trained model saved as {final_model_path}")

    # Calculate and print the overall accuracy
    overall_accuracy = np.mean(accuracies)
    print(f"\nOverall Cross-Validation Accuracy: {overall_accuracy * 100:.2f}%")

    # Calculate and print the confusion matrix
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    print("\nConfusion Matrix:")
    print(cm)

    print("Cross-validation completed.")

In [21]:
if __name__ == "__main__":
    main()

Processing Audio: 100%|██████████| 1191/1191 [01:32<00:00, 12.92it/s]


Epoch 1/20
30/30 [==============================] - 8s 168ms/step - loss: 1.5062 - accuracy: 0.5714 - val_loss: 1.1815 - val_accuracy: 0.6695
Epoch 2/20
30/30 [==============================] - 5s 156ms/step - loss: 1.4472 - accuracy: 0.5914 - val_loss: 2.1575 - val_accuracy: 0.6444
Epoch 3/20
30/30 [==============================] - 4s 145ms/step - loss: 1.3352 - accuracy: 0.6208 - val_loss: 1.2944 - val_accuracy: 0.6987
Epoch 4/20
30/30 [==============================] - 5s 157ms/step - loss: 1.2463 - accuracy: 0.6187 - val_loss: 1.0243 - val_accuracy: 0.7364
Epoch 5/20
30/30 [==============================] - 5s 164ms/step - loss: 1.1713 - accuracy: 0.5956 - val_loss: 1.4661 - val_accuracy: 0.6234
Epoch 6/20
30/30 [==============================] - 5s 154ms/step - loss: 0.9987 - accuracy: 0.6639 - val_loss: 1.1949 - val_accuracy: 0.7029
Epoch 7/20
30/30 [==============================] - 4s 144ms/step - loss: 1.1264 - accuracy: 0.6197 - val_loss: 0.8462 - val_accuracy: 0.6862
Epoch 

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 35ms/step
Fold 1 - Validation Accuracy: 76.15%
Epoch 1/20
30/30 [==============================] - 7s 154ms/step - loss: 1.4005 - accuracy: 0.5960 - val_loss: 3.8431 - val_accuracy: 0.5840
Epoch 2/20
30/30 [==============================] - 4s 141ms/step - loss: 1.5856 - accuracy: 0.6055 - val_loss: 1.0622 - val_accuracy: 0.6471
Epoch 3/20
30/30 [==============================] - 5s 151ms/step - loss: 1.3639 - accuracy: 0.5929 - val_loss: 1.3455 - val_accuracy: 0.6429
Epoch 4/20
30/30 [==============================] - 4s 144ms/step - loss: 1.1137 - accuracy: 0.6264 - val_loss: 1.5468 - val_accuracy: 0.6387
Epoch 5/20
30/30 [==============================] - 4s 142ms/step - loss: 1.0143 - accuracy: 0.6632 - val_loss: 0.8893 - val_accuracy: 0.7227
Epoch 6/20
30/30 [==============================] - 4s 149ms/step - loss: 1.1531 - accuracy: 0.5971 - val_loss: 0.8364 - val_accuracy: 0.7143
Epoch 7/20
30/30 [==============================] - 5s 154m

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 30ms/step
Fold 4 - Validation Accuracy: 71.01%
Epoch 1/20
30/30 [==============================] - 8s 175ms/step - loss: 1.5284 - accuracy: 0.5656 - val_loss: 1.7489 - val_accuracy: 0.7353
Epoch 2/20
30/30 [==============================] - 5s 154ms/step - loss: 1.7133 - accuracy: 0.5981 - val_loss: 1.3914 - val_accuracy: 0.7437
Epoch 3/20
30/30 [==============================] - 5s 155ms/step - loss: 1.7954 - accuracy: 0.5929 - val_loss: 3.8006 - val_accuracy: 0.5714
Epoch 4/20
30/30 [==============================] - 5s 149ms/step - loss: 1.2831 - accuracy: 0.6128 - val_loss: 3.2257 - val_accuracy: 0.5924
Epoch 5/20
30/30 [==============================] - 4s 144ms/step - loss: 1.5457 - accuracy: 0.5887 - val_loss: 3.2383 - val_accuracy: 0.5588


/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 35ms/step
Fold 5 - Validation Accuracy: 74.37%

Final trained model saved as final_model/final_model_20240117_094006.h5

Overall Cross-Validation Accuracy: 73.13%

Confusion Matrix:
[[462 173]
 [147 409]]
Cross-validation completed.


/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
